In [ ]:
track = 'https://envirocar.org/api/stable/tracks/63948bc3ad53a0015a08780f'
streets = 'https://overpass-api.de/api/interpreter?data=[out%3Ajson][timeout%3A25]%3B%0A(%0A%20%20way["highway"~"^(motorway|trunk|primary|residential|tertiary|motorway_link|unclassified|service|secondary|secondary_link|trunk|trunk_link)%24"](51.45%2C7.268%2C51.4531%2C7.274)%3B%0A)%3B%0Aout%20geom%3B'

In [ ]:
pip install geopandas

In [ ]:
pip install contextily


In [ ]:
pip install pyproj

In [ ]:
pip install scipy

In [ ]:
import geopandas as gp
from shapely.geometry import shape
import requests
import contextily as cx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.spatial import distance
import json
import time

In [ ]:
response = requests.get(streets)
mystreets = response.json()['elements']
for d in mystreets:
    coords = []
    for p in d['geometry']:
        coords.append((p['lon'],p['lat']))
    data = {"type": "LineString", "coordinates": coords}
    d['geometry'] = shape(data)
gdf = gp.GeoDataFrame(mystreets).set_geometry('geometry')
#gdf.head()

In [ ]:
mystreets

In [ ]:
mytrack = gp.read_file(track)
#mytrack

In [ ]:
tmp_list = []
for x in range(0,mytrack.shape[0],4):
    tmp_list.append({
      'id': mytrack.loc[x,'id'],
      'time': mytrack.loc[x,'time'],
      'phenomenons': mytrack.loc[x,'phenomenons'],
      'geometry': mytrack.loc[x,'geometry']
     })
subTracks = gp.GeoDataFrame(tmp_list)
#subTracks

In [ ]:
#subTracks

In [ ]:
# check PROJ_LIB env variable. it should point to rasterio specific prof_data folder
# make sure that system env variable points to that respective location
echo %PROJ_LIB%

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower
#worldMap = gp.read_file(gp.datasets.get_path("naturalearth_lowres"))
ax = subTracks.plot(ax=gdf.plot(), color='red', markersize=8)
cx.add_basemap(ax, source=cx.providers.OpenStreetMap.Mapnik, crs=4326)

In [ ]:
puffer = subTracks.buffer(0.0002)

In [ ]:
puffergdf = gp.GeoDataFrame(geometry=puffer)

In [ ]:
puffergdf.insert(0, "Id", range(1,1+len(puffergdf)))

In [ ]:
tempStreets2 = gp.GeoDataFrame()
for puff in puffergdf.iterrows():
    tempStreets = []
    tempStreets = gp.overlay(gdf, gp.GeoDataFrame(geometry=[puff[1]['geometry']]), how='intersection')
    tempStreets['pufferId'] = puff[1]['Id']
    tempStreets2 = pd.concat([tempStreets2,tempStreets])

In [ ]:
import shapely

In [ ]:
snappedPoints = []
i=0
for index, row in tempStreets2.iterrows():
    activePoint = subTracks.loc[row['pufferId']-1]
    snappedPoints.append({
        'id': i,
        'pufferId': row['pufferId'],
        'pointId': activePoint['id'],
        'geometry': row['geometry'].interpolate(row['geometry'].project(activePoint.geometry))
     })
    i = i+1
snappedPointsGdf = gp.GeoDataFrame(snappedPoints)

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower
#worldMap = gp.read_file(gp.datasets.get_path("naturalearth_lowres"))
ax = subTracks.plot(ax=snappedPointsGdf.plot(), color='red', markersize=8)
cx.add_basemap(ax, source=cx.providers.OpenStreetMap.Mapnik, crs=4326)

In [ ]:
#snappedPointsGdf

In [ ]:
koordinaten = {}
for index, row in snappedPointsGdf.iterrows():
    koordinaten[row['id']] = (row['geometry'].x, row['geometry'].y)
koordinaten

In [ ]:
#dict(list(koordinaten.items())[:5])

In [ ]:
#puffergdf

In [ ]:
init_graph1 = {}
lastIndex = -1
for index, puffrow in puffergdf.iterrows():
    if lastIndex != -1: #nicht erste iteration
        tempPoints = snappedPointsGdf.loc[snappedPointsGdf['pufferId'] == puffrow['Id']]
        for j, firstCol in (snappedPointsGdf.loc[snappedPointsGdf['pufferId'] == lastIndex]).iterrows():
            init_graph1[firstCol['id']] = list(tempPoints['id'])
    lastIndex = puffrow['Id']
#init_graph1

In [ ]:
def getDistances(sourcePs, destPs, metrics = "duration"):
    # bei "duration" antwort in Sekunden
    # dei "distance" antwort in Metern
    url = "https://ors5.fbg-hsbo.de//v2/matrix/driving-car"
    N = len(sourcePs)
    M = len(destPs)
    locationsP = sourcePs + destPs
    body = {"locations":locationsP, "destinations":list(range(N,N+M)), "sources":list(range(0,N)), "metrics":[metrics]}
    # spalten nach destinations, zeile nach sources
    headers = {'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf624837b0be5d89604065b03e13c82ba84c06',
    'Content-Type': 'application/json; charset=utf-8'}
    call = requests.post(url, json=body)
    edgeValues = json.loads(call.text)[metrics+"s"]
    return edgeValues

In [ ]:
def createForks(sourcePIDs, destPIDs, distMatrix):
    retDict = {}
    for sPID in sourcePIDs:
        retDict[sPID] = {}
    for j in range(0, len(sourcePIDs)):
        sourcePID = sourcePIDs[j]
        for i in range(0, len(destPIDs)):
            #print(i)
            retDict[sourcePID][destPIDs[i]] = distMatrix[j][i]
    return retDict

In [ ]:
def createDistGraph(pDict, forkDict, metrics = "duration"):
    outputGraph = {}
    for sourcePID, destPIDList in forkDict.items():
        tempDestPDict = {}
        for destPID in destPIDList:
            tempDestPDict[destPID] = pDict[destPID]
            pDict[sourcePID]
            outputGraph.update(createForks([sourcePID],list(tempDestPDict.keys()),getDistances([pDict[sourcePID]],list(tempDestPDict.values()), metrics = metrics)))
    return outputGraph

In [ ]:
def createProbGraph(pDict, forkDict, metrics = "duration"):
    #TODO: Kantenwerte als Wahrscheinlichkeioten...
    outputGraph = {}
    for sourcePID, destPIDList in forkDict.items():
        tempDestPDict = {}
        for destPID in destPIDList:
            tempDestPDict[destPID] = pDict[destPID]
            pDict[sourcePID]
            distances = getDistances([pDict[sourcePID]],list(tempDestPDict.values()), metrics = metrics)
            total = sum(distances[0])                         
            ws = []
            for value in distances[0]:
                ws.append(value/total)
            outputGraph.update(createForks([sourcePID],list(tempDestPDict.keys()),[ws]))
            #time.sleep(10)
    return outputGraph

In [ ]:
outputGraph = createProbGraph(koordinaten, init_graph1)

In [ ]:
#outputGraph

In [ ]:
d_graph = {}
d_graph[-1] = {}
for index, node in snappedPointsGdf.iterrows():
    d_graph[node['id']] = {}
lastIndex = -1
for index, puffrow in puffergdf.iterrows():
    tempPoints = snappedPointsGdf.loc[snappedPointsGdf['pufferId'] == puffrow['Id']]
    summe = 0
    for index, tp1 in tempPoints.iterrows():
        actSnappedPoint = (tp1['geometry'].x, tp1['geometry'].y)
        actBasePoint = (subTracks['geometry'].iloc[puffrow['Id']-1].x, subTracks['geometry'].iloc[puffrow['Id']-1].y)
        abstand = distance.euclidean(actSnappedPoint, actBasePoint)
        summe = summe + abstand
    if lastIndex == -1:
        for index, tp in tempPoints.iterrows():
            actSnappedPoint = (tp['geometry'].x, tp['geometry'].y)
            actBasePoint = (subTracks['geometry'].iloc[puffrow['Id']-1].x, subTracks['geometry'].iloc[puffrow['Id']-1].y)
            abstand = distance.euclidean(actSnappedPoint, actBasePoint)
            d_graph[-1][tp['id']] = 1-(abstand/summe)
    else:
        for i, secondCol in tempPoints.iterrows():
            for j, firstCol in (snappedPointsGdf.loc[snappedPointsGdf['pufferId'] == lastIndex]).iterrows():
                actBasePoint = (subTracks['geometry'].iloc[puffrow['Id']-1].x, subTracks['geometry'].iloc[puffrow['Id']-1].y)
                secColPoint = (secondCol['geometry'].x, secondCol['geometry'].y)
                abstand = distance.euclidean(actBasePoint, secColPoint)
                #d_graph[firstCol['id']][secondCol['id']] = (1-(abstand/summe)) * 100
                #print(abstand/summe)
                d_graph[firstCol['id']][secondCol['id']] = (1-(abstand/summe)) * outputGraph[firstCol['id']][secondCol['id']]
    lastIndex = puffrow['Id']
for index, pts in snappedPointsGdf.loc[snappedPointsGdf['pufferId'] == puffergdf['Id'].max()].iterrows():
    d_graph[pts['id']][-2] = 0

In [ ]:
#d_graph

In [ ]:
#els = list(d_graph.items())
#els[-1]

In [ ]:
#d_graph[-1]

In [ ]:
unbesuchteKnoten = list(d_graph)
unbesuchteKnoten.append(-2)
tabelle = pd.DataFrame(unbesuchteKnoten, columns=['Knoten'])
tabelle['Vorgaenger'] = -99
tabelle['Gesamtdistanz'] = float('inf')
startID = -1
endID = -2
#tabelle

In [ ]:
tabelle.loc[tabelle['Knoten'] == -1,'Gesamtdistanz'] = 0
for nachbarn in d_graph[startID].items():
    tabelle.loc[tabelle['Knoten'] == nachbarn[0],'Gesamtdistanz'] = nachbarn[1]
    tabelle.loc[tabelle['Knoten'] == nachbarn[0],'Vorgaenger'] = -1
unbesuchteKnoten.remove(-1)
while (len(unbesuchteKnoten) != 0 and (-2 in unbesuchteKnoten)):
    tempTabelle = tabelle[tabelle['Vorgaenger'] != -99]
    tempTabelle.sort_values(by=['Gesamtdistanz'], inplace=True)
    i =0
    while(tempTabelle.iloc[i]['Knoten'] not in unbesuchteKnoten):
        i = i+1
    activePoint = tempTabelle.iloc[i]
    if (activePoint['Knoten'] != -2):
        for nachbarn in d_graph[activePoint['Knoten']].items():
            if nachbarn[0] in unbesuchteKnoten:
                if ((activePoint['Gesamtdistanz']+nachbarn[1]) < tabelle[tabelle['Knoten'] == nachbarn[0]]['Gesamtdistanz']).bool():
                    tabelle.loc[tabelle['Knoten'] == nachbarn[0],'Gesamtdistanz'] = activePoint['Gesamtdistanz']+nachbarn[1]
                    tabelle.loc[tabelle['Knoten'] == nachbarn[0],'Vorgaenger'] = activePoint['Knoten']
            tabelle
    unbesuchteKnoten.remove(activePoint['Knoten'])
#tabelle

In [ ]:
weg = [-2]
while ((tabelle[tabelle["Knoten"] == weg[-1]]["Vorgaenger"]).item() != -99):
    weg.append(tabelle[tabelle["Knoten"] == weg[-1]]["Vorgaenger"].item())
weg.reverse()
weg.remove(-1)
weg.remove(-2)
weg

In [ ]:
wegkoordinaten = []
for id in weg:
    wegkoordinaten.append([koordinaten[id][0], koordinaten[id][1]])
wegkoordinaten

In [ ]:
body = {"coordinates":wegkoordinaten}
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf624837b0be5d89604065b03e13c82ba84c06',
    'Content-Type': 'application/json; charset=utf-8'
}
call = requests.post('https://api.openrouteservice.org/v2/directions/driving-car/geojson', json=body, headers=headers)
wegpoints = json.loads(call.text)["features"][0]["geometry"]
print(wegpoints)

In [ ]:
geom = [shape(i) for i in [wegpoints]]
fahrtwegGDF = gp.GeoDataFrame({'geometry':geom})
fahrtwegGDF

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower
#worldMap = gp.read_file(gp.datasets.get_path("naturalearth_lowres"))
ax = subTracks.plot(ax=fahrtwegGDF.plot(), color='red', markersize=8)
cx.add_basemap(ax, source=cx.providers.OpenStreetMap.Mapnik, crs=4326)

In [ ]:
tmp_list = []
for x in range(0,mytrack.shape[0],1):
    tmp_list.append({
      'id': mytrack.loc[x,'id'],
      'time': mytrack.loc[x,'time'],
      'phenomenons': mytrack.loc[x,'phenomenons'],
      'geometry': mytrack.loc[x,'geometry']
     })
myTrack2 = gp.GeoDataFrame(tmp_list)
myTrack2

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower
#worldMap = gp.read_file(gp.datasets.get_path("naturalearth_lowres"))
ax = myTrack2.plot(ax=gdf.plot(), color='red', markersize=8)
cx.add_basemap(ax, source=cx.providers.OpenStreetMap.Mapnik, crs=4326)